In [ ]:
from bs4 import BeautifulSoup
import re
from collections import Counter

In [ ]:
with open('Nibelungenlied_A_TEI.txt', 'r') as f:
  text = f.read()

In [ ]:
def find_characters(text):
  soup = BeautifulSoup(text, 'lxml')
  characters_in_part = []
  for character in soup.find_all('rs'):
    characters_in_part.append(character.attrs['ref'])

  return characters_in_part

In [ ]:
def clean_text_part(text):
  text = re.sub(r'[.,!?\-]', '', text)
  clean_original_name = re.sub(r'(<rs[^>]*>)(.*?)(</rs>)', r'\1\3', text)
  clean_from_tags = clean_original_name.replace('<rs ref="', '')
  clean_from_tags = clean_from_tags.replace('"></rs>', '')

  return clean_from_tags

In [ ]:
def pairs_in_part(text):

  text_clean = clean_text_part(text).split()
  characters = find_characters(text)

  pairs = []
  for i in range(len(text_clean)):
    if text_clean[i] in characters:
      window = text_clean[i+1:min(i+20, len(text_clean))] #чтобы не переваливало за границу части, если окно будет больше
      for element in window:
        if element in characters and element != text_clean[i]:
          pair = min(text_clean[i], element) + '_' + max(text_clean[i], element)
          pairs.append(pair)

  return pairs

In [ ]:
text_parts = text.split('———————————')

In [ ]:
all_pairs = []

In [ ]:
for element in text_parts:
  all_pairs = all_pairs + pairs_in_part(element)

In [ ]:
counts = Counter(all_pairs)

In [ ]:
def write_result(counts, filename):
    csv = 'Source,Target,Weight' + '\n'

    for count in counts:
        if counts[count] >= 2:
            word1 = count.split('_')[0]
            word2 = count.split('_')[1]
            weight = str(counts[count])
            csv += word1 + ',' + word2 + ',' + weight + '\n'

    with open(filename, 'w') as f:
        f.write(csv)

In [ ]:
write_result(counts, 'ner_graph_nieb_A.csv')